Import Lib

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from datetime import datetime

Get data realtime

In [2]:
url = 'https://bmkg-content-inatews.storage.googleapis.com/live30event.xml'
resp = urllib.request.urlopen(url)
xml_data = resp.read()
root = ET.fromstring(xml_data)

Make a data list

In [3]:
event_list =[]
status_list =[]
waktu_list = []
lintang_list = []
bujur_list = []
dalam_list =[]
mag_list =[]
fokal_list =[]
area_list = []

Print data realtime 

In [4]:
# Get first element 'Gempa'
gempa = root.find('gempa')

# Get all element 'person'
for gempa in root.findall('gempa'):
    event = gempa.find('eventid').text
    status = gempa.find('status').text
    waktu = gempa.find('waktu').text
    # waktus= datetime.strptime(waktu, "%Y/%m/%d %H:%M:%S.%f")
    lintang = gempa.find('lintang').text
    bujur = gempa.find('bujur').text
    dalam = gempa.find('dalam').text
    mag = gempa.find('mag').text
    fokal = gempa.find('fokal').text
    area = gempa.find('area').text
    #Input data into list
    event_list.append(event) 
    status_list.append(status)
    waktu_list.append(waktu)
    lintang_list.append(lintang)
    bujur_list.append(bujur)
    dalam_list.append(dalam)
    mag_list.append(mag) 
    fokal_list.append(fokal)
    area_list.append(area)

Tampilkan data list ke data frame

In [5]:
import pandas as pd

In [6]:
#input list of data into dataframe
event_list
status_list
waktu_list 
lintang_list 
bujur_list 
dalam_list
mag_list
fokal_list
area_list 

datas = pd.DataFrame({'ID Event':event_list, 'Status Gempa':status_list, 'Waktu Gempa':waktu_list, 'Lintang Gempa':lintang_list,
                    'Bujur Gempa':bujur_list, 'Kedalaman Gempa':dalam_list, 'Magnitudo Gempa':mag_list, 'Fokal Gempa':fokal_list,
                    'Area Gempa':area_list,})
datas

,ID Event,Status Gempa,Waktu Gempa,Lintang Gempa,Bujur Gempa,Kedalaman Gempa,Magnitudo Gempa,Fokal Gempa,Area Gempa
0,bmg2023mzhj,confirmed,2023/07/04 14:33:15.003,-0.17,123.02,34,2.9,undetermined,"Minahassa Peninsula, Sulawesi"
1,bmg2023mzhe,confirmed,2023/07/04 14:27:13.750,-3.52,130.00,42,2.9,undetermined,"Seram, Indonesia"
2,bmg2023mzge,confirmed,2023/07/04 13:57:02.111,-3.87,102.46,10,3.7,undetermined,"Southern Sumatra, Indonesia"
3,bmg2023mzgb,confirmed,2023/07/04 13:53:36.194,-7.75,107.32,32,2.5,undetermined,"Java, Indonesia"
4,bmg2023mzef,confirmed,2023/07/04 12:57:32.180,4.85,96.08,10,3.3,undetermined,"Northern Sumatra, Indonesia"
...,...,...,...,...,...,...,...,...,...
191,bmg2023mpll,confirmed,2023/06/29 05:15:48.132,-20.40,-178.03,506,5.3,undetermined,Fiji Islands Region
192,bmg2023mpky,confirmed,2023/06/29 04:57:59.417,37.27,71.58,148,5.7,undetermined,Afghanistan-Tajikistan Border Region
193,bmg2023mpht,confirmed,2023/06/29 03:23:16.421,-9.21,119.60,22,2.9,undetermined,"Sumba Region, Indonesia"
194,bmg2023mphm,confirmed,2023/06/29 03:14:32.885,-7.52,122.38,10,3.2,undetermined,Flores Sea


INPUT DATA TO DATABASE PROCESS

In [7]:
import mysql.connector

make database

In [8]:
# db = mysql.connector.connect(
#     host = "localhost",
#     user = "root",
#     passwd = "",
# )

# cursor = db.cursor()
# cursor.execute("CREATE DATABASE gempa")

# print("Database gempa berhasil dibuat")

Connect with database made before

In [9]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database ="gempa"
)

if db.is_connected():
  print("Berhasil terhubung ke database gempa")
else:
  print("Cek XAMPP terlebih dahulu!")

Berhasil terhubung ke database gempa


Create Table 

In [ ]:
db = mysql.connector.connect(
   host = "localhost",
   user = "root",
   passwd = "",
   database = "gempa"
)

cursor = db.cursor()

table1 = """CREATE TABLE info(
   id_event VARCHAR(20),
   status VARCHAR(10),
   waktu TIMESTAMP,
   lintang DOUBLE,
   garis_bujur DOUBLE,
   kedalaman DOUBLE,
   magnitudo DOUBLE,
   fokal VARCHAR (15), 
   area VARCHAR (100)
   )"""

table2 = """CREATE TABLE infogempa(
   id_event VARCHAR(20),
   status VARCHAR(10),
   waktu TIMESTAMP PRIMARY KEY,
   lintang DOUBLE,
   garis_bujur DOUBLE,
   kedalaman DOUBLE,
   magnitudo DOUBLE,
   fokal VARCHAR (15), 
   area VARCHAR (100)
   )"""

cursor.execute(table1)
cursor.execute(table2)

Input data to temporary database

In [11]:
cursor = db.cursor()
datas
nama_tabel = "info"

for i, row in datas.iterrows():
    id_event = row['ID Event']
    status = row['Status Gempa']
    waktu = row['Waktu Gempa']
    lintang = row['Lintang Gempa']
    garis_bujur = row['Bujur Gempa']
    kedalaman = row['Kedalaman Gempa']
    magnitudo = row['Magnitudo Gempa']
    fokal = row['Fokal Gempa']
    area = row['Area Gempa']

    sql = "INSERT INTO {} (id_Event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(nama_tabel)
    val = (id_event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area)
    cursor.execute(sql, val)
    db.commit()


Input to Fix Database

In [ ]:
# cursor = db.cursor()
# datas
# nama_tabel = "infogempa"

# for i, row in datas.iterrows():
#     id_event = row['ID Event']
#     status = row['Status Gempa']
#     waktu = row['Waktu Gempa']
#     lintang = row['Lintang Gempa']
#     garis_bujur = row['Bujur Gempa']
#     kedalaman = row['Kedalaman Gempa']
#     magnitudo = row['Magnitudo Gempa']
#     fokal = row['Fokal Gempa']
#     area = row['Area Gempa']

#     sql = "INSERT INTO {} (id_Event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(nama_tabel)
#     val = (id_event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area)
#     cursor.execute(sql, val)
#     db.commit()

Try to input one of data realtime

In [ ]:
# cursor = db.cursor()
# nama_tabel = "infogempa"   

# sql = "INSERT INTO {} (id_Event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(nama_tabel)
# val = (event_list[0], status_list[0], waktu_list[0], lintang_list[0],bujur_list[0], dalam_list[0], mag_list[0], fokal_list[0], area_list[0])
# cursor.execute(sql, val)
# db.commit()

Database check

In [ ]:
cursor = db.cursor()
sql = "SELECT * FROM info"
cursor.execute(sql)

results = cursor.fetchall()
for data in results:
  print(data)

('bmg2023masf', 'confirmed', datetime.datetime(2023, 6, 21, 3, 37, 52), -8.2, 117.64, 10.0, 2.6, 'undetermined', 'Sumbawa Region, Indonesia')
('bmg2023mamp', 'confirmed', datetime.datetime(2023, 6, 21, 0, 47, 49), -0.32, 123.12, 56.0, 4.6, 'undetermined', 'Minahassa Peninsula, Sulawesi')
('bmg2023mals', 'confirmed', datetime.datetime(2023, 6, 21, 0, 20, 38), -8.03, 113.57, 10.0, 2.3, 'undetermined', 'Java, Indonesia')
('bmg2023majj', 'confirmed', datetime.datetime(2023, 6, 20, 23, 9, 40), -6.11, 104.03, 52.0, 3.9, 'undetermined', 'Sunda Strait, Indonesia')
('bmg2023maip', 'confirmed', datetime.datetime(2023, 6, 20, 22, 46, 40), -4.91, 103.06, 25.0, 3.3, 'undetermined', 'Southern Sumatra, Indonesia')
('bmg2023maif', 'confirmed', datetime.datetime(2023, 6, 20, 22, 35), -4.68, 121.96, 10.0, 3.1, 'undetermined', 'Sulawesi, Indonesia')
('bmg2023mahd', 'confirmed', datetime.datetime(2023, 6, 20, 22, 2, 30), -8.74, 125.18, 63.0, 3.3, 'undetermined', 'Timor Region')
('bmg2023madx', 'confirmed'

In [ ]:
# cursor = db.cursor()
# sql = "SELECT * FROM infogempa"
# cursor.execute(sql)

# results = cursor.fetchall()
# for data in results:
#   print(data)

Close Database Connection

In [ ]:
db.close()
print("KONEKSI DITUTUP")

KONEKSI DITUTUP
